In [ ]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, Birch, MiniBatchKMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import hdbscan
from minisom import MiniSom
import numpy as np

# Create clster

In [ ]:
def add_clusters(df: pd.DataFrame, features: list[str], method: str, cluster_col: str = None, **kwargs) -> pd.DataFrame:
    """
    Добавляет кластерные метки в DataFrame без повторного масштабирования (предполагается, что данные уже нормализованы).
    """
    X = df[features].values  # используем как есть
    method = method.lower()
    cluster_col = cluster_col or f"cluster_{method}"

    if method == "kmeans":
        model = KMeans(**kwargs)
        labels = model.fit_predict(X)

    elif method == "dbscan":
        model = DBSCAN(**kwargs)
        labels = model.fit_predict(X)

    elif method == "agglomerative":
        model = AgglomerativeClustering(**kwargs)
        labels = model.fit_predict(X)

    elif method == "gmm":
        model = GaussianMixture(**kwargs)
        labels = model.fit_predict(X)

    elif method == "birch":
        model = Birch(**kwargs)
        labels = model.fit_predict(X)

    elif method == "hdbscan":
        import hdbscan
        model = hdbscan.HDBSCAN(**kwargs)
        labels = model.fit_predict(X)

    elif method == "spectral":
        model = SpectralClustering(**kwargs)
        labels = model.fit_predict(X)

    elif method == "minibatch_kmeans":
        model = MiniBatchKMeans(**kwargs)
        labels = model.fit_predict(X)

    elif method == "som":
        from minisom import MiniSom
        som_x = kwargs.get("x", 4)
        som_y = kwargs.get("y", 4)
        sigma = kwargs.get("sigma", 1.0)
        learning_rate = kwargs.get("learning_rate", 0.5)
        som = MiniSom(som_x, som_y, X.shape[1], sigma=sigma, learning_rate=learning_rate)
        som.random_weights_init(X)
        som.train_random(X, 100)
        labels = [som.winner(x)[0] * som_y + som.winner(x)[1] for x in X]

    else:
        raise ValueError(f"Unknown clustering method: {method}")

    df = df.copy()
    df[cluster_col] = labels
    return df


# Safe cluster

In [ ]:
df = pd.read_csv("scoring_data.csv")

features = ["income", "loan_amount", "age"]

df_kmeans = add_clusters(df, features, method="kmeans", n_clusters=4)

df_dbscan = add_clusters(df, features, method="dbscan", eps=0.5, min_samples=5)

df_hdb = add_clusters(df, features, method="hdbscan", min_cluster_size=10)

df_som = add_clusters(df, features, method="som", x=4, y=4)
